<a href="https://colab.research.google.com/github/adt125/AI_project_2023/blob/master/Rule_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import nltk 
import numpy as np
import re
import random
from nltk.stem import wordnet                                  # to perform lemmitization
from sklearn.feature_extraction.text import CountVectorizer    # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer    # to perform tfidf
from nltk import pos_tag                                       # for parts of speech
from sklearn.metrics import pairwise_distances                 # to perfrom cosine similarity
from nltk import word_tokenize                                 # to create tokens
from nltk.corpus import stopwords                              # for stop words
import nltk
import pickle
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [24]:
df = pd.read_csv("mentalhealth.csv")
df.head()

,Question_ID,Questions,Answers
0,1590140,What is mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,"Mental illness does can affect anyone, regardl..."
2,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
3,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."
4,1619387,What should I do if I know someone who appears...,We encourage those with symptoms to talk to th...


In [7]:
nltk.download('punkt') 
nltk.download('wordnet')
lemma = wordnet.WordNetLemmatizer() 
# lemma.lemmatize('passed', pos='v') #example

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
nltk.download('averaged_perceptron_tagger') 
nltk.download('stopwords')
stop = stopwords.words('english')
pickle.dump(stop,open('stop.pkl','wb'))
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
# function that performs text normalization steps and returns the lemmatized tokens as a sentence

def text_normalization(text):
    text = str(text).lower()                        # text to lower case
    spl_char_text = re.sub(r'[^ a-z]','',text)      # removing special characters
    tokens = nltk.word_tokenize(spl_char_text)      # word tokenizing
    lema = wordnet.WordNetLemmatizer()              # intializing lemmatization
    tags_list = pos_tag(tokens,tagset=None)         # parts of speech
    lema_words = []                                 # empty list 
    for token,pos_token in tags_list:               # lemmatize according to POS
        if pos_token.startswith('V'):               # Verb
            pos_val = 'v'
        elif pos_token.startswith('J'):             # Adjective
            pos_val = 'a'
        elif pos_token.startswith('R'):             # Adverb
            pos_val = 'r'
        else:
            pos_val = 'n'                           # Noun
        lema_token = lema.lemmatize(token,pos_val)

        if lema_token in stop: 
          lema_words.append(lema_token)             # appending the lemmatized token into a list
    
    return " ".join(lema_words) 

In [26]:
df['lemmatized_text'] = df['Questions'].apply(text_normalization)   # clean text
df.head(5)

,Question_ID,Questions,Answers,lemmatized_text
0,1590140,What is mental illness?,Mental illnesses are health conditions that di...,what be
1,2110618,Who does mental illness affect?,"Mental illness does can affect anyone, regardl...",who do
2,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...,what be some of the of
3,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi...",can with
4,1619387,What should I do if I know someone who appears...,We encourage those with symptoms to talk to th...,what should i do if i who to have the of a


In [27]:
cv = CountVectorizer()                                  # intializing the count vectorizer
X = cv.fit_transform(df['lemmatized_text']).toarray()

In [31]:

# returns all the unique word from data 

features = cv.get_feature_names_out()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()

,about,after,an,and,any,be,before,between,but,can,...,to,too,we,what,when,where,who,why,with,you
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0


In [35]:
# using tf-idf

tfidf = TfidfVectorizer()                                             # intializing tf-id 
x_tfidf = tfidf.fit_transform(df['lemmatized_text']).toarray()        # transforming the data into array

In [32]:
df_tfidf = pd.DataFrame(x_tfidf,columns = tfidf.get_feature_names_out())
df_tfidf.head()

,about,after,an,and,any,be,before,between,but,can,...,to,too,we,what,when,where,who,why,with,you
0,0.0,0.0,0.0,0.0,0.0,0.746943,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.664888,0.0,0.0,0.000000,0.0,0.00000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.874033,0.0,0.00000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.240038,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.213669,0.0,0.0,0.000000,0.0,0.00000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.407601,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.91316,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.314706,0.0,0.0,0.205709,0.0,0.0,0.467509,0.0,0.00000,0.0


In [ ]:
import pickle

In [17]:
pickle.dump(df_tfidf,open('df_tdidf.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

In [18]:
def chat_tfidf(text):
    lemma = text_normalization(text) # calling the function to perform text normalization
    tf = tfidf.transform([lemma]).toarray() # applying tf-idf
    cos = 1-pairwise_distances(df_tfidf,tf,metric='cosine') # applying cosine similarity
    index_value = cos.argmax() # getting index value 
    return df['Answers'].loc[index_value]

In [19]:
chat_tfidf('what causes mental health issues?')

'Challenges or problems with your mental health can arise from psychological, biological, and social, issues, as well as life events.'

In [20]:
chat_tfidf('How can I feel better?')

'What you tell yourself about a situation affects how you feel and what you do. Sometimes your interpretation of a situation can get distorted and you only focus on the negative aspects—this is normal and expected. However, when you interpret situations too negatively, you might feel worse. You\'re also more likely to respond to the situation in ways that are unhelpful in the long term. \n These automatic thoughts and assumptions are sometimes called thinking traps. Everyone falls into unbalanced thinking traps from time to time. You\'re most likely to distort your interpretation of things when you already feel sad, angry, anxious, depressed or stressed. You\'re also more vulnerable to thinking traps when you\'re not taking good care of yourself, like when you\'re not eating or sleeping well. \n Here are some common thinking traps: \n Thinking that a negative situation is part of a constant cycle of bad things that happen. People who overgeneralize often use words like "always" or "nev

In [ ]:

# defining a function that returns response to query using bow

def chat_bow(text):
    lemma = text_normalization(text) # calling the function to perform text normalization
    bow = cv.transform([lemma]).toarray() # applying bow
    cosine_value = 1- pairwise_distances(df_bow,bow, metric = 'cosine' )
    index_value = cosine_value.argmax() # getting index value 
    return df['Answers'].loc[index_value]

In [21]:
greet_inp=["hello", "hi", "what's up?","hey" ]
greet_responses=["Hi!","Hello!","Hey there"]
def greet(sentence):
  for word in sentence.split():
    if word.lower() in greet_inp:
      return random.choice(greet_responses)

In [36]:
flag=True
print("Mental Health FAQ")
while(flag==True):
  user_response=input()
  user_response=user_response.lower()
  if(user_response!='bye'):
    if(user_response=='thanks' or user_response=='thank you'):
      flag=False
      print("You're welcome")
    else:
      if(greet(user_response)!=None):
        print(greet(user_response))
      else:
        print(chat_tfidf(user_response))
  else:
      flag=False
print("Goodbye! Take Care <3")


Mental Health FAQ
hi
Hi!
what is metnal illness
Mental illnesses are health conditions that disrupt a person's thoughts, emotions, relationships, and daily functioning.
what is anxiety
Mental illnesses are health conditions that disrupt a person's thoughts, emotions, relationships, and daily functioning.
what is the meaning of anxiety
There are many types of mental health professionals. Finding the right one for you may require some research.
who is a professional
Mental illness does can affect anyone, regardless of gender, age, income, social status, ethnicity, religion, sexual orientation, or background.
who can help
Mental illness does can affect anyone, regardless of gender, age, income, social status, ethnicity, religion, sexual orientation, or background.
help and suppport
Positive and negative symptoms are medical terms for two groups of symptoms in schizophrenia. 
 Positive symptoms add. Positive symptoms include hallucinations (sensations that aren’t real), delusions (beliefs 